In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as pyplot
import os
import re
import math
from enum import Enum

# Pomožne funkcije

Globalne spremenljivke za direktorij z vhodnimi slikami in direktorij, kjer se bodo shranile izhodne slike.

V direktorij, ki je definiran z `ASSETS_FOLDER` se iz [te povezave](https://scihub.copernicus.eu/dhus/#/home) ročno dobijo in shranijo vhodne slike, ki morajo biti tipa `.jp2`.
V direktorij, ki je definiran z `OUTPUT_FOLDER` pa se bodo shranli izračunani indeksi.

Oba direktorija morata obstajati za pravilno delovanje programa.

In [ ]:
ASSETS_FOLDER = "assets/"
OUTPUT_FOLDER = "output/"

Enum razred, ki vsebuje vse podprte indekse.

In [ ]:
class Index(Enum):
  EVI = 'evi',
  NDVI = 'ndvi',
  GNDVI = 'gndvi',
  MSI = 'msi',
  NDWI = 'ndwi',
  NDBI = 'ndbi',
  NDMI = 'ndmi',
  NBR = 'nbr',
  MACCIONI = 'maccioni',
  PVR = 'pvr',
  DATT1 = 'datt1',
  SIPI1 = 'sipi1'

Pomožna funkcija za prikaz sivinske slike.

In [ ]:
def display_image(image, title):
  imgplot = pyplot.imshow(image, cmap="gray")
  pyplot.axis('off')
  pyplot.title(title)
  pyplot.show()

Ker so slike precej velike, se ta funkcija sprehodi čez vse datoteke v direktoriju `ASSETS_FOLDER` in s pomočjo regexa shrani vse ujemajoče se datoteke v slovarju, ki je uporabljen ob izračunu indeksov. Vse slike se ne shranijo v pomnilnik, ampak se uporabijo le takrat, ko so zahtevane v izračunu indeksa.

In [ ]:
def setup_dictionary(folder):
  images = {}
  for filename in os.listdir(folder):
    key = re.search('B[0|1|8][0-9|A]', filename)
    if key:      
      images[key.group(0)] = filename

      print('handled:', key.group(0))
  return images

Slike so normalizirane na interval `[0, 1]`, vendar lahko pri normalizaciji pride do napak, zato se vse vrednosti, ki so izven tega intervala popravijo na ustrezne vrednosti.

In [ ]:
def fix_image(image):
  image[image > 1] = 1
  image[image <= 0] = 0
  return image

Funkcija, ki s naloži sliko iz diska in jo normalizira.

In [ ]:
def load_image(filename):
  image = cv2.imread(ASSETS_FOLDER + filename, -1)
  max_value = np.iinfo(image.dtype).max

  if image is not None:
    image = image.astype(np.float32)
    image = image / max_value
    image = fix_image(image)

    return image

Funkcija, ki prejme sliko, velikost slike in faktor skaliranja. Če je faktor skaliranja večji od 1, potem mora slika biti povečana na ustrezno velikost, in sicer njena velikost se pomnoži s faktorjem skaliranja. Če je faktor skaliranja enak 1, potem ni potrebno povečevati slike in funkcija ne stori ničesar.

In [ ]:
def upscale_image(image, size, factor):
  if factor == 1:
    return image
    
  print('upscaling by: ' + str(factor))
  image = cv2.resize(image, (size * factor, size * factor))
  return image

Spodnja funkcija prejme niz slik, ki so uporabljene v izračunu posameznega indeksa. Pridobijo se vse velikosti slik in pridobi se tudi maksimalna velikost izmed teh slik. Funkcija potem za vsako sliko izvede skaliranje in popravljanje slike, če je potrebno.

In [ ]:
def upscale_images(images):
  shapes = list(map(lambda image: image.shape[0], images))
  max_size = max(shapes)

  for id, image in enumerate(images):
    images[id] = upscale_image(image, image.shape[0], math.floor(max_size / image.shape[0]))
    images[id] = fix_image(images[id])

  return images

Funkcija za izračun indeksov. Indeksi so bili izračunani po spodnjih formulah.

Pet poljubnih indeksov je bilo **naključno** izbranih iz [te spletne strani](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/indexdb/)

### Indeksi iz predavanj

$ EVI = \frac{\vec{s}[8]\;-\;\vec{s}[4]}{\vec{s}[8]\;+\;6\vec{s}[4]\;-\;7.5\vec{s}[2]\;+\;1} $

$ NDVI = \frac{\vec{s}[8]\;-\;\vec{s}[4]}{\vec{s}[8]\;+\;\vec{s}[4]} $

$ GNDVI = \frac{\vec{s}[8]\;-\;\vec{s}[3]}{\vec{s}[8]\;+\;\vec{s}[3]} $

$ MSI = \frac{\vec{s}[11]}{\vec{s}[8]} $

$ NDWI = \frac{\vec{s}[3]\;-\;\vec{s}[11]}{\vec{s}[3]\;+\;\vec{s}[11]} $

$ NDBI = \frac{\vec{s}[11]\;-\;\vec{s}[8]}{\vec{s}[11]\;+\;\vec{s}[8]} $

$ NDMI = \frac{\vec{s}[9]\;-\;\vec{s}[8]}{\vec{s}[9]\;+\;\vec{s}[8]} $


### Pet poljubnih indeksov

$ NBR = \frac{\vec{s}[8]\;-\;\vec{s}[12]}{\vec{s}[8]\;+\;\vec{s}[12]} $

$ Maccioni = \frac{\vec{s}[7]\;-\;\vec{s}[5]}{\vec{s}[7]\;-\;\vec{s}[4]} $

$ PVR = \frac{\vec{s}[3]\;-\;\vec{s}[4]}{\vec{s}[3]\;+\;\vec{s}[4]} $

$ Datt1 = \frac{\vec{s}[8]\;-\;\vec{s}[5]}{\vec{s}[8]\;-\;\vec{s}[4]} $

$ SIPI1 = \frac{\vec{s}[8]\;-\;\vec{s}[1]}{\vec{s}[8]\;-\;\vec{s}[4]} $

In [ ]:
def calculate_index(images, index):
  print('calculating:', index.name)

  if index == Index.EVI:
    loaded_images = [load_image(images['B08']), load_image(images['B04']), load_image(images['B02'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + 6 * loaded_images[1] - 7.5 * loaded_images[2] + 1
    
    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.NDVI:
    loaded_images = [load_image(images['B08']), load_image(images['B04'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.GNDVI:
    loaded_images = [load_image(images['B08']), load_image(images['B03'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.MSI:
    loaded_images = [load_image(images['B11']), load_image(images['B08'])]
    loaded_images = upscale_images(loaded_images)

    return np.divide(loaded_images[0], loaded_images[1], out=np.zeros_like(loaded_images[0]), where=loaded_images[1]!=0)
  elif index == Index.NDWI:
    loaded_images = [load_image(images['B03']), load_image(images['B11'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.NDBI:
    loaded_images = [load_image(images['B11']), load_image(images['B08'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.NDMI:
    loaded_images = [load_image(images['B09']), load_image(images['B08'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.NBR:
    loaded_images = [load_image(images['B08']), load_image(images['B12'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.MACCIONI:
    loaded_images = [load_image(images['B07']), load_image(images['B05']), load_image(images['B04'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[2]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.PVR:
    loaded_images = [load_image(images['B03']), load_image(images['B04'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] + loaded_images[1]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.DATT1:
    loaded_images = [load_image(images['B08']), load_image(images['B05']), load_image(images['B04'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[2]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  elif index == Index.SIPI1:
    loaded_images = [load_image(images['B08']), load_image(images['B01']), load_image(images['B04'])]
    loaded_images = upscale_images(loaded_images)
    numerator = loaded_images[0] - loaded_images[1]
    denominator = loaded_images[0] - loaded_images[2]

    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
  else:
    return None

# Glavni del programa

V tem kratkem odseku se izvede inicializacija slovarja.

In [ ]:
images = setup_dictionary(ASSETS_FOLDER)

Nato se prikaže vsak izmed pasov.

In [ ]:
for key in images:
  image = load_image(images[key])
  print(key + ': ' + images[key] + ' ' + str(image.shape))
  
  display_image(image, key)

Na koncu pa je `for` zanka, ki gre čez vse indekse v Enum razredu in rezultate shrani na disk.

In [ ]:
for index in Index:
  index_image = calculate_index(images, index)

  if index_image is not None:
    print('saving to:', OUTPUT_FOLDER + index.name + ".tif")

    cv2.imwrite(OUTPUT_FOLDER + index.name + ".tif", index_image)

# Rezultati

Na spodnji sliki se nahajajo izračunani indeksi, v tem primeru so bili uporabljeni podatki iz okolice Reykjavika.

![](docs/calculated.PNG)